### Board Utilities

In [2]:
import random
import math

# --- Board Functions ---
def create_board():
    return [' ' for _ in range(9)]

def print_board(board):
    print()
    for i in range(3):
        row = []
        for j in range(3):
            idx = i*3 + j
            row.append(board[idx] if board[idx] != ' ' else str(idx))
        print(' ' + ' | '.join(row))
        if i < 2:
            print("-----------")
    print()

def available_moves(board):
    return [i for i, spot in enumerate(board) if spot == ' ']

def make_move(board, position, letter):
    board[position] = letter

def winner(board, letter):
    win_patterns = [
        [0, 1, 2], [3, 4, 5], [6, 7, 8],
        [0, 3, 6], [1, 4, 7], [2, 5, 8],
        [0, 4, 8], [2, 4, 6]
    ]
    return any(all(board[i] == letter for i in pattern) for pattern in win_patterns)

def is_full(board):
    return ' ' not in board

### AI Algorithms

#### 1. Random Move

In [5]:
def ai_random(board):
    return random.choice(available_moves(board))

#### 2. Heuristic (center > corners > others)

In [7]:
def ai_heuristic(board):
    if 4 in available_moves(board):
        return 4
    corners = [i for i in [0, 2, 6, 8] if i in available_moves(board)]
    if corners:
        return random.choice(corners)
    return ai_random(board)

#### 3. Minimax Algorithm

In [9]:
def minimax(board, depth, is_maximizing, ai_letter, human_letter):
    if winner(board, ai_letter):
        return 10 - depth
    elif winner(board, human_letter):
        return depth - 10
    elif is_full(board):
        return 0

    if is_maximizing:
        best_score = -math.inf
        for move in available_moves(board):
            board[move] = ai_letter
            score = minimax(board, depth + 1, False, ai_letter, human_letter)
            board[move] = ' '
            best_score = max(score, best_score)
        return best_score
    else:
        best_score = math.inf
        for move in available_moves(board):
            board[move] = human_letter
            score = minimax(board, depth + 1, True, ai_letter, human_letter)
            board[move] = ' '
            best_score = min(score, best_score)
        return best_score

def ai_minimax(board, ai_letter, human_letter):
    best_score = -math.inf
    best_move = None
    for move in available_moves(board):
        board[move] = ai_letter
        score = minimax(board, 0, False, ai_letter, human_letter)
        board[move] = ' '
        if score > best_score:
            best_score = score
            best_move = move
    return best_move

#### 4. Monte-Carlo Tree Search(MCTS) Algorithm

In [11]:
def simulate_random_game(board, current_letter):
    letters = ['X', 'O']
    while True:
        if winner(board, 'X'):
            return 'X'
        if winner(board, 'O'):
            return 'O'
        if is_full(board):
            return None
        move = random.choice(available_moves(board))
        board[move] = current_letter
        current_letter = letters[1] if current_letter == letters[0] else letters[0]

def ai_mcts(board, ai_letter, simulations=100):
    moves = available_moves(board)
    move_scores = {move: 0 for move in moves}

    for move in moves:
        for _ in range(simulations):
            temp_board = board[:]
            temp_board[move] = ai_letter
            winner_letter = simulate_random_game(
                temp_board,
                'O' if ai_letter == 'X' else 'X'
            )
            if winner_letter == ai_letter:
                move_scores[move] += 1
            elif winner_letter is None:
                move_scores[move] += 0.5

    best_move = max(moves, key=lambda m: move_scores[m])
    return best_move

### AI Selection Helper

In [13]:
def choose_ai():
    print("Choose AI technique:")
    print("1. Random Move")
    print("2. Heuristic")
    print("3. Minimax")
    print("4. Monte Carlo Tree Search")
    return int(input("Enter choice (1-4): "))

def get_ai_move(ai_choice, board, letter, opponent):
    if ai_choice == 1:
        return ai_random(board)
    elif ai_choice == 2:
        return ai_heuristic(board)
    elif ai_choice == 3:
        return ai_minimax(board, letter, opponent)
    elif ai_choice == 4:
        return ai_mcts(board, letter)

### Game Modes

In [15]:
def pvp_mode():
    board = create_board()
    current_letter = 'X'
    while True:
        print_board(board)
        move = int(input(f"Player {current_letter}, enter move (0-8): "))
        if move not in available_moves(board):
            print("Invalid move! Try again.")
            continue
        make_move(board, move, current_letter)

        if winner(board, current_letter):
            print_board(board)
            print(f"Player {current_letter} wins!")
            break
        if is_full(board):
            print_board(board)
            print("It's a draw!")
            break

        current_letter = 'O' if current_letter == 'X' else 'X'

def pvai_mode():
    board = create_board()
    human_letter = input("Choose your letter (X/O): ").upper()
    ai_letter = 'O' if human_letter == 'X' else 'X'
    ai_choice = choose_ai()
    current_letter = 'X'

    while True:
        print_board(board)
        if current_letter == human_letter:
            move = int(input(f"Player {current_letter}, enter move (0-8): "))
        else:
            print(f"AI ({current_letter}) is thinking...")
            move = get_ai_move(ai_choice, board, ai_letter, human_letter)

        if move not in available_moves(board):
            print("Invalid move! Try again.")
            continue
        make_move(board, move, current_letter)

        if winner(board, current_letter):
            print_board(board)
            print(f"{current_letter} wins!")
            break
        if is_full(board):
            print_board(board)
            print("It's a draw!")
            break

        current_letter = 'O' if current_letter == 'X' else 'X'

def aivai_mode():
    board = create_board()
    print("Select AI for X:")
    ai_x_choice = choose_ai()
    print("Select AI for O:")
    ai_o_choice = choose_ai()
    current_letter = 'X'

    while True:
        print_board(board)
        if current_letter == 'X':
            move = get_ai_move(ai_x_choice, board, 'X', 'O')
        else:
            move = get_ai_move(ai_o_choice, board, 'O', 'X')

        make_move(board, move, current_letter)

        if winner(board, current_letter):
            print_board(board)
            print(f"AI ({current_letter}) wins!")
            break
        if is_full(board):
            print_board(board)
            print("It's a draw!")
            break

        current_letter = 'O' if current_letter == 'X' else 'X'

### Main Runner

In [17]:
def play_game():
    print("Select mode:")
    print("1. Player vs Player")
    print("2. Player vs AI")
    print("3. AI vs AI")
    mode = int(input("Enter mode: "))

    if mode == 1:
        pvp_mode()
    elif mode == 2:
        pvai_mode()
    elif mode == 3:
        aivai_mode()

# Run this cell to start the game
play_game()

Select mode:
1. Player vs Player
2. Player vs AI
3. AI vs AI


Enter mode:  3


Select AI for X:
Choose AI technique:
1. Random Move
2. Heuristic
3. Minimax
4. Monte Carlo Tree Search


Enter choice (1-4):  2


Select AI for O:
Choose AI technique:
1. Random Move
2. Heuristic
3. Minimax
4. Monte Carlo Tree Search


Enter choice (1-4):  3



 0 | 1 | 2
-----------
 3 | 4 | 5
-----------
 6 | 7 | 8


 0 | 1 | 2
-----------
 3 | X | 5
-----------
 6 | 7 | 8


 O | 1 | 2
-----------
 3 | X | 5
-----------
 6 | 7 | 8


 O | 1 | 2
-----------
 3 | X | 5
-----------
 6 | 7 | X


 O | 1 | O
-----------
 3 | X | 5
-----------
 6 | 7 | X


 O | 1 | O
-----------
 3 | X | 5
-----------
 X | 7 | X


 O | O | O
-----------
 3 | X | 5
-----------
 X | 7 | X

AI (O) wins!
